<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 10


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

Создать базовый класс Invoice в C#, который будет представлять информацию о
фактурах за поставленные товары или оказанные услуги. На основе этого класса
разработать 2-3 производных класса, демонстрирующих принципы наследования и
полиморфизма. В каждом из классов должны быть реализованы новые атрибуты и
методы, а также переопределены некоторые методы базового класса для
демонстрации полиморфизма.
Требования к базовому классу Invoice:
• Атрибуты: Номер фактуры (InvoiceNumber), Дата выдачи (IssueDate), Общая
сумма (TotalAmount).
• Методы:
o
o CalculateTotal(): метод для расчета общей суммы по фактуре.
o AddLine(LineItem lineItem): метод для добавления позиции в фактуру.
o RemoveLine(LineItem lineItem): метод для удаления позиции из
фактуры.
Требования к производным классам:
1. ТоварнаяФактура (GoodsInvoice): Должна содержать дополнительные
атрибуты, такие как Дата поставки (SupplyDate). Метод AddLine() должен
быть переопределен для добавления информации о дате поставки товара
при добавлении позиции.
2. УслуговаяФактура (ServiceInvoice): Должна содержать дополнительные
атрибуты, такие как Дата оказания услуги (ServiceDate).
Метод RemoveLine() должен быть переопределен для добавления
информации о причине аннулирования услуги при удалении позиции.
3. КомбинированнаяФактура (CombinedInvoice) (если требуется третий класс):
Должна содержать дополнительные атрибуты, такие как Наличие возврата
(ReturnAllowed). Метод CalculateTotal() должен быть переопределен для
учета возможного возврата товара или услуги при расчете общей суммы.

#### Дополнительное задание
Добавьте к сущестующим классам конструктора классов с использованием гетторов и сетторов и реализуйте взаимодействие объектов между собой

<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [1]:
public interface IInvoice
{
    void CalculateTotal();
    void AddLine(LineItem lineItem);
    void RemoveLine(string description);
}

public class Invoice : IInvoice
{
    public string _invoiceNumber; //Номер фактуры
    public DateTime _issueDate; // Дата выдачи
    public decimal _totalAmount; // Общая сумма
    protected List<LineItem> LineItems { get; set; } // Список товаров на фактуре

    public string InvoiceNumber
    {
        get{return _invoiceNumber;}
        set{_invoiceNumber = value;}
    }

    public DateTime IssueDate
    {
        get{return _issueDate;}
        set{_issueDate = value;}
    }

    public decimal TotalAmount
    {
        get{return _totalAmount;}
        set{
            if(_totalAmount>=0)
                _totalAmount = value;
            else
            throw new ArgumentOutOfRangeException("Общая сумма не может быть отрицательной");

        }
    }


    public Invoice(string _invoiceNumber, DateTime _issueDate) // инициализация
    {
        InvoiceNumber = _invoiceNumber;
        IssueDate = _issueDate;
        LineItems = new List<LineItem>();
    }

    public virtual void CalculateTotal() // метод для подсчета суммы товаров на фактуре
    {
        TotalAmount = 0;
        foreach (var item in LineItems)
        {
            TotalAmount += item.Amount;
        }
    }

    public virtual void AddLine(LineItem lineItem) // метод добавления товаров на фактуру, а затем пересчитывает сумму
    {
        LineItems.Add(lineItem);
        CalculateTotal();
    }

    public virtual void RemoveLine(string description) // Удаление товара с фактуры, с перерасчетом
    {
        var lineItem = LineItems.FirstOrDefault(item => item.Description == description);
        if (lineItem != null)
        {
            LineItems.Remove(lineItem);
            CalculateTotal();
        }
    }
}

public class LineItem
{
    public string _description;// Описание позиции
    public decimal _amount;// Стоимость

    public string Description
    {
        get{return _description;}
        set{_description = value;}
    }

    public decimal Amount
    {
        get{return _amount;}
        set{
            if(_amount>=0)
                _amount = value;
            else
            throw new ArgumentOutOfRangeException("Стоимость не может быть отрицательной");

        }
    }

    public LineItem(string _description, decimal _amount) 
    {
        Description = _description;
        Amount = _amount;
    }
}

public class GoodsInvoice : Invoice
{
    public DateTime _supplyDate;// дата поставки

    public DateTime SupplyDate
    {
        get{return _supplyDate;}
        set{_supplyDate = value;}
    }

    // base - вызов конструктора родителя для производного класса, для инициализации необходимых полей и тд.
    public GoodsInvoice(string _invoiceNumber, DateTime _issueDate, DateTime _supplyDate): base(_invoiceNumber, _issueDate) 
    {
        SupplyDate = _supplyDate;
    }

    public override void AddLine(LineItem lineItem) // добавляет элемент с указанием даты доставки
    {
        base.AddLine(lineItem);
        Console.WriteLine($"Товар добавлен с датой поставки: {SupplyDate}");
    }
}

public class ServiceInvoice : Invoice
{
    public DateTime _serviceDate; // дата оказания услуги

    public DateTime ServiceDate
    {
        get{return _serviceDate;}
        set{_serviceDate = value;}
    }

    public ServiceInvoice(string _invoiceNumber, DateTime _issueDate, DateTime _serviceDate): base(_invoiceNumber, _issueDate)
    {
        ServiceDate = _serviceDate;
    }

    public override void RemoveLine(string description) // удаляет элемент из списка, с указанием причины
    {
        base.RemoveLine(description);
        Console.WriteLine("Услуга была отменена. Пожалуйста, укажите причину.");
    }
}

public class CombinedInvoice : Invoice
{
    public bool ReturnAllowed { get; } // флаг, который определяет, разрешен ли возврат

    public CombinedInvoice(string invoiceNumber, DateTime issueDate, bool returnAllowed) : base(invoiceNumber, issueDate)
    {
        ReturnAllowed = returnAllowed;
    }

    public override void RemoveLine(string description)
    {
        if(ReturnAllowed)
        {
            base.RemoveLine(description);
            Console.WriteLine($"Товар {description} возвращен, сумма пересчитана");
        }
        else
        {
            Console.WriteLine($"Товар {description} не подлежит возрату");
        }
    }
}


// Создаем инвойс товаров и добавляем строки с товарами.
       GoodsInvoice myGoodsInvoice = new GoodsInvoice("100", DateTime.Now, DateTime.Now.AddDays(5));
       myGoodsInvoice.AddLine(new LineItem("Товар 1", 150));
       myGoodsInvoice.AddLine(new LineItem("Товар 2", 250));
       Console.WriteLine($"Общая сумма товаров: {myGoodsInvoice.TotalAmount}");

       // Создаем инвойс услуг и добавляем строки с услугами.
       ServiceInvoice myServiceInvoice = new ServiceInvoice("101", DateTime.Now, DateTime.Now.AddDays(1));
       myServiceInvoice.AddLine(new LineItem("Услуга 1", 150));
       myServiceInvoice.AddLine(new LineItem("Услуга 2", 250));
       myServiceInvoice.AddLine(new LineItem("Услуга 3", 300));
       Console.WriteLine($"Общая сумма услуг: {myServiceInvoice.TotalAmount}");
       
       myServiceInvoice.RemoveLine("Услуга 2"); // Удаляем услугу и выводим новую сумму.
       Console.WriteLine($"Общая сумма услуг после удаления: {myServiceInvoice.TotalAmount}");

       // Создаем комбинированный инвойс и добавляем строки.
       CombinedInvoice myCombinedInvoice = new CombinedInvoice("102", DateTime.Now, true);
       myCombinedInvoice.AddLine(new LineItem("Товар A", 250));
       myCombinedInvoice.AddLine(new LineItem("Товар B", 270));
       myCombinedInvoice.AddLine(new LineItem("Товар C", 300));
       Console.WriteLine($"Общая сумма комбинированного инвойса: {myCombinedInvoice.TotalAmount}");
       
       myCombinedInvoice.RemoveLine("Товар B"); // Возвращаем товар и выводим новую сумму.
       Console.WriteLine($"Общая сумма комбинированного инвойса после возврата: {myCombinedInvoice.TotalAmount}");

Товар добавлен с датой поставки: 11/13/2024 3:04:18 PM
Товар добавлен с датой поставки: 11/13/2024 3:04:18 PM
Общая сумма товаров: 400
Общая сумма услуг: 700
Услуга была отменена. Пожалуйста, укажите причину.
Общая сумма услуг после удаления: 450
Общая сумма комбинированного инвойса: 820
Товар Товар B возвращен, сумма пересчитана
Общая сумма комбинированного инвойса после возврата: 550
